# Package Import

In [1]:
import torch
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torchvision
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import ast
import pandas as pd
import os
from PIL import Image
from kor2vec import Kor2Vec # Kor2Vec import

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

dfroot='/home/augustin/project/bitamin-conference/data/crawling/data/dataframe'


# Concat Dataframe

In [3]:
# bottom info
a = pd.read_csv(dfroot + '/bottom_cotton_info.csv', encoding='UTF-8')
a["categories"] = 'cotton'
b = pd.read_csv(dfroot+ '/bottom_denim_info.csv', encoding='UTF-8')
b["categories"] = "denim"
c = pd.read_csv(dfroot + '/bottom_etc_info.csv', encoding='UTF-8')
c["categories"] = "etc"
d = pd.read_csv(dfroot + '/bottom_jogger_info.csv', encoding='UTF-8')
d["categories"] = "jogger"
e = pd.read_csv(dfroot + '/bottom_jumpper_info.csv', encoding='UTF-8')
e["categories"] = "jumpper"
f = pd.read_csv(dfroot + '/bottom_leggings_info.csv', encoding='UTF-8')
f["categories"] = "leggings"
g = pd.read_csv(dfroot + '/bottom_short_info.csv', encoding='UTF-8')
g["categories"] = "short"
h = pd.read_csv(dfroot + '/bottom_slacks_info.csv', encoding='UTF-8')
h["categories"] = "slacks"

# top info
a1 = pd.read_csv(dfroot +'/top_hood_info.csv', encoding='UTF-8')
a1["categories"] = 'hood'
b1 = pd.read_csv(dfroot +'/top_knit_info.csv', encoding='UTF-8')
b1["categories"] = "knit"
c1 = pd.read_csv(dfroot +'/top_long_info.csv', encoding='UTF-8')
c1["categories"] = "long"
d1 = pd.read_csv(dfroot +'/top_polo_info.csv', encoding='UTF-8')
d1["categories"] = "polo"
e1 = pd.read_csv(dfroot +'/top_shirts_info.csv', encoding='UTF-8')
e1["categories"] = "shirts"
f1 = pd.read_csv(dfroot +'/top_short_info.csv', encoding='UTF-8')
f1["categories"] = "short"
g1 = pd.read_csv(dfroot +'/top_sleeveless_info.csv', encoding='UTF-8')
g1["categories"] = "sleeveless"
h1 = pd.read_csv(dfroot +'/top_sweatshirt_info.csv', encoding='UTF-8')
h1["categories"] = "sweatshirt"

img_dir= '/home/augustin/project/bitamin-conference/data/crawling/data/image/'
model_dir = "/home/augustin/project/bitamin-conference/recsys/model/modelfile/"
kor_vec_name= "embedding_final_1"

total_df = pd.concat([a,b,c,d,e,f,g,h,a1,b1,c1,d1,e1,f1,g1,h1], axis=0)
total_df.reset_index(inplace=True)
total_df.drop(columns = ["index"], inplace=True)
total_df

,Unnamed: 0,product_num,product_name,brand,year_sold,like,rate,price,tag,categories
0,0,2861160,데일리 코듀로이 밴딩 팬츠 - 4 COLOR,SUARE,1.6만 개 이상,"19,150",4.8,"59,000원","바지,팬츠,와이드팬츠,밴딩팬츠,코듀로이,코튼팬츠,캐주얼팬츠",cotton
1,1,957401,P[1003]롱 와이드 코튼팬츠(베이지),PUBLIC ICON,1.8만 개 이상,"55,445",4.7,"68,000원","데님팬츠,와이드면팬츠,와이드코튼팬츠,와이드팬츠,와이드,베이지",cotton
2,2,2060759,[기모선택] 와이드 절개 벌룬팬츠 코튼 카키,GLW,1.6만 개 이상,"35,679",4.9,"74,800원","와이드팬츠,코튼팬츠,힙합,와이드핏,하이웨이스트,오버핏,아메카지",cotton
3,3,1627892,레플리카 퍼티그 팬츠 [카키],MUSINSA STANDARD,1.6만 개 이상,"14,833",4.9,"43,900원","밀리터리,레플리카,군용,포켓,백사틴,바지,면바지,캐주얼팬츠,유아인착용상품",cotton
4,4,2183795,에센셜 세미 와이드 코듀로이 팬츠 크림,GLW,1.1만 개 이상,"21,312",4.8,"49,000원","와이드팬츠,겨울바지,골덴팬츠,골덴바지,면바지,남자오버핏,코듀로이팬츠",cotton
...,...,...,...,...,...,...,...,...,...,...
44169,6082,1602281,해리 피그먼트 워싱 맨투맨 카키 OYMT9058,UNDERBASE,50개 이상,111,4.7,"37,900원","스웨트셔츠,맨투맨,오버핏,피그먼트,로고맨투맨,오버핏맨투맨",sweatshirt
44170,6083,1580282,MARITHE OVAL LOGO SWEATSHIRT black,MARITHE FRANCOIS GIRBAUD,50개 미만,111,4.6,"79,000원","맨투맨,스웻셔츠,로고맨투맨,스웻,MFG맨투맨,엠에프지맨투맨,스웨트셔츠",sweatshirt
44171,6084,1319747,엠엘 모어 이즈 레스 레터링 크루넥 스웨트 [멜란지],THEMILE,50개 미만,44,4.9,"44,600원","데일리,맨투맨,오버핏,커플,크루넥",sweatshirt
44172,6085,1241808,토트넘맨투맨 TTNRMA03 [BLACK],TOTTENHAM APPAREL,50개 미만,49,4.6,"49,000원","나라물산,맨투맨,오버핏,토트넘손흥민",sweatshirt


## Add columns (filename)

In [4]:
def add_image_filenames(df):
    filenames = []
    for row_index in df.index:
        each_files = []
        temp_num=df.loc[row_index]['product_num']
        each_files.append(os.path.join(img_dir+f'{temp_num}.jpg'))
        filenames.append(each_files)
    # Add new column
    df['image_filename'] = list(filenames)

    return df


total_df = add_image_filenames(total_df)
total_df

,Unnamed: 0,product_num,product_name,brand,year_sold,like,rate,price,tag,categories,image_filename
0,0,2861160,데일리 코듀로이 밴딩 팬츠 - 4 COLOR,SUARE,1.6만 개 이상,"19,150",4.8,"59,000원","바지,팬츠,와이드팬츠,밴딩팬츠,코듀로이,코튼팬츠,캐주얼팬츠",cotton,[/home/augustin/project/bitamin-conference/dat...
1,1,957401,P[1003]롱 와이드 코튼팬츠(베이지),PUBLIC ICON,1.8만 개 이상,"55,445",4.7,"68,000원","데님팬츠,와이드면팬츠,와이드코튼팬츠,와이드팬츠,와이드,베이지",cotton,[/home/augustin/project/bitamin-conference/dat...
2,2,2060759,[기모선택] 와이드 절개 벌룬팬츠 코튼 카키,GLW,1.6만 개 이상,"35,679",4.9,"74,800원","와이드팬츠,코튼팬츠,힙합,와이드핏,하이웨이스트,오버핏,아메카지",cotton,[/home/augustin/project/bitamin-conference/dat...
3,3,1627892,레플리카 퍼티그 팬츠 [카키],MUSINSA STANDARD,1.6만 개 이상,"14,833",4.9,"43,900원","밀리터리,레플리카,군용,포켓,백사틴,바지,면바지,캐주얼팬츠,유아인착용상품",cotton,[/home/augustin/project/bitamin-conference/dat...
4,4,2183795,에센셜 세미 와이드 코듀로이 팬츠 크림,GLW,1.1만 개 이상,"21,312",4.8,"49,000원","와이드팬츠,겨울바지,골덴팬츠,골덴바지,면바지,남자오버핏,코듀로이팬츠",cotton,[/home/augustin/project/bitamin-conference/dat...
...,...,...,...,...,...,...,...,...,...,...,...
44169,6082,1602281,해리 피그먼트 워싱 맨투맨 카키 OYMT9058,UNDERBASE,50개 이상,111,4.7,"37,900원","스웨트셔츠,맨투맨,오버핏,피그먼트,로고맨투맨,오버핏맨투맨",sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44170,6083,1580282,MARITHE OVAL LOGO SWEATSHIRT black,MARITHE FRANCOIS GIRBAUD,50개 미만,111,4.6,"79,000원","맨투맨,스웻셔츠,로고맨투맨,스웻,MFG맨투맨,엠에프지맨투맨,스웨트셔츠",sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44171,6084,1319747,엠엘 모어 이즈 레스 레터링 크루넥 스웨트 [멜란지],THEMILE,50개 미만,44,4.9,"44,600원","데일리,맨투맨,오버핏,커플,크루넥",sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44172,6085,1241808,토트넘맨투맨 TTNRMA03 [BLACK],TOTTENHAM APPAREL,50개 미만,49,4.6,"49,000원","나라물산,맨투맨,오버핏,토트넘손흥민",sweatshirt,[/home/augustin/project/bitamin-conference/dat...


## Tag preprocess

In [5]:
class preprocess(Dataset):
    def to_list(x):
        return ast.literal_eval(x)
    
    def max_len(x):
        return len(x)

    def split_(x):
        return x.replace(',',' ')
    
    
    total_df['tag'] = total_df['tag'].apply(split_)

In [6]:
total_df

,Unnamed: 0,product_num,product_name,brand,year_sold,like,rate,price,tag,categories,image_filename
0,0,2861160,데일리 코듀로이 밴딩 팬츠 - 4 COLOR,SUARE,1.6만 개 이상,"19,150",4.8,"59,000원",바지 팬츠 와이드팬츠 밴딩팬츠 코듀로이 코튼팬츠 캐주얼팬츠,cotton,[/home/augustin/project/bitamin-conference/dat...
1,1,957401,P[1003]롱 와이드 코튼팬츠(베이지),PUBLIC ICON,1.8만 개 이상,"55,445",4.7,"68,000원",데님팬츠 와이드면팬츠 와이드코튼팬츠 와이드팬츠 와이드 베이지,cotton,[/home/augustin/project/bitamin-conference/dat...
2,2,2060759,[기모선택] 와이드 절개 벌룬팬츠 코튼 카키,GLW,1.6만 개 이상,"35,679",4.9,"74,800원",와이드팬츠 코튼팬츠 힙합 와이드핏 하이웨이스트 오버핏 아메카지,cotton,[/home/augustin/project/bitamin-conference/dat...
3,3,1627892,레플리카 퍼티그 팬츠 [카키],MUSINSA STANDARD,1.6만 개 이상,"14,833",4.9,"43,900원",밀리터리 레플리카 군용 포켓 백사틴 바지 면바지 캐주얼팬츠 유아인착용상품,cotton,[/home/augustin/project/bitamin-conference/dat...
4,4,2183795,에센셜 세미 와이드 코듀로이 팬츠 크림,GLW,1.1만 개 이상,"21,312",4.8,"49,000원",와이드팬츠 겨울바지 골덴팬츠 골덴바지 면바지 남자오버핏 코듀로이팬츠,cotton,[/home/augustin/project/bitamin-conference/dat...
...,...,...,...,...,...,...,...,...,...,...,...
44169,6082,1602281,해리 피그먼트 워싱 맨투맨 카키 OYMT9058,UNDERBASE,50개 이상,111,4.7,"37,900원",스웨트셔츠 맨투맨 오버핏 피그먼트 로고맨투맨 오버핏맨투맨,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44170,6083,1580282,MARITHE OVAL LOGO SWEATSHIRT black,MARITHE FRANCOIS GIRBAUD,50개 미만,111,4.6,"79,000원",맨투맨 스웻셔츠 로고맨투맨 스웻 MFG맨투맨 엠에프지맨투맨 스웨트셔츠,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44171,6084,1319747,엠엘 모어 이즈 레스 레터링 크루넥 스웨트 [멜란지],THEMILE,50개 미만,44,4.9,"44,600원",데일리 맨투맨 오버핏 커플 크루넥,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44172,6085,1241808,토트넘맨투맨 TTNRMA03 [BLACK],TOTTENHAM APPAREL,50개 미만,49,4.6,"49,000원",나라물산 맨투맨 오버핏 토트넘손흥민,sweatshirt,[/home/augustin/project/bitamin-conference/dat...


In [7]:
temp =set()
for image_file in os.listdir(os.path.join(img_dir))[:]:
    temp.add(image_file)
    
import re
a = set()
for string in temp:
    numbers = re.findall(r'\d+', string)
    a.add(int(*numbers))
    
b = set(total_df.product_num) #상품번호

final = a&b

## 사진에만있는거추출
total_df = total_df.query("product_num in @ final")
total_df.reset_index(inplace=True)
total_df.drop(columns = ["index"], inplace=True)
total_df

,Unnamed: 0,product_num,product_name,brand,year_sold,like,rate,price,tag,categories,image_filename
0,0,2861160,데일리 코듀로이 밴딩 팬츠 - 4 COLOR,SUARE,1.6만 개 이상,"19,150",4.8,"59,000원",바지 팬츠 와이드팬츠 밴딩팬츠 코듀로이 코튼팬츠 캐주얼팬츠,cotton,[/home/augustin/project/bitamin-conference/dat...
1,1,957401,P[1003]롱 와이드 코튼팬츠(베이지),PUBLIC ICON,1.8만 개 이상,"55,445",4.7,"68,000원",데님팬츠 와이드면팬츠 와이드코튼팬츠 와이드팬츠 와이드 베이지,cotton,[/home/augustin/project/bitamin-conference/dat...
2,2,2060759,[기모선택] 와이드 절개 벌룬팬츠 코튼 카키,GLW,1.6만 개 이상,"35,679",4.9,"74,800원",와이드팬츠 코튼팬츠 힙합 와이드핏 하이웨이스트 오버핏 아메카지,cotton,[/home/augustin/project/bitamin-conference/dat...
3,3,1627892,레플리카 퍼티그 팬츠 [카키],MUSINSA STANDARD,1.6만 개 이상,"14,833",4.9,"43,900원",밀리터리 레플리카 군용 포켓 백사틴 바지 면바지 캐주얼팬츠 유아인착용상품,cotton,[/home/augustin/project/bitamin-conference/dat...
4,4,2183795,에센셜 세미 와이드 코듀로이 팬츠 크림,GLW,1.1만 개 이상,"21,312",4.8,"49,000원",와이드팬츠 겨울바지 골덴팬츠 골덴바지 면바지 남자오버핏 코듀로이팬츠,cotton,[/home/augustin/project/bitamin-conference/dat...
...,...,...,...,...,...,...,...,...,...,...,...
44169,6082,1602281,해리 피그먼트 워싱 맨투맨 카키 OYMT9058,UNDERBASE,50개 이상,111,4.7,"37,900원",스웨트셔츠 맨투맨 오버핏 피그먼트 로고맨투맨 오버핏맨투맨,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44170,6083,1580282,MARITHE OVAL LOGO SWEATSHIRT black,MARITHE FRANCOIS GIRBAUD,50개 미만,111,4.6,"79,000원",맨투맨 스웻셔츠 로고맨투맨 스웻 MFG맨투맨 엠에프지맨투맨 스웨트셔츠,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44171,6084,1319747,엠엘 모어 이즈 레스 레터링 크루넥 스웨트 [멜란지],THEMILE,50개 미만,44,4.9,"44,600원",데일리 맨투맨 오버핏 커플 크루넥,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44172,6085,1241808,토트넘맨투맨 TTNRMA03 [BLACK],TOTTENHAM APPAREL,50개 미만,49,4.6,"49,000원",나라물산 맨투맨 오버핏 토트넘손흥민,sweatshirt,[/home/augustin/project/bitamin-conference/dat...


## Image to vector

In [8]:
# # Load the pretrained model
# model = models.resnet18(pretrained=True)
# # Use the model object to select the desired layer
# layer = model._modules.get('avgpool')

# # Set model to evaluation mode
# model.eval()
# scaler = transforms.Resize((224, 224))
# normalize = transforms.Normalize(mean=[0.5,0.5,0.5],
#                                  std=[0.5,0.5,0.5])
# to_tensor = transforms.ToTensor()

In [9]:
# def get_vector(image_name):
#     # 1. Load the image with Pillow library
#     img = Image.open(image_name)
#     # 2. Create a PyTorch Variable with the transformed image
#     t_img = Variable(normalize(to_tensor(scaler(img))).unsqueeze(0))

#     # 3. Create a vector of zeros that will hold our feature vector
#     #    The 'avgpool' layer has an output size of 512
#     my_embedding = torch.zeros(512)
#     # 4. Define a function that will copy the output of a layer
#     def copy_data(m, i, o):
#         my_embedding.copy_(o.data.reshape(o.data.size(1)))
#     # 5. Attach that function to our selected layer
#     h = layer.register_forward_hook(copy_data)
#     # 6. Run the model on our transformed image
#     model(t_img)
#     # 7. Detach our copy function from the layer
#     h.remove()
#     # 8. Return the feature vector
#     return my_embedding.cpu().detach().numpy()

In [10]:
# for i,file in enumerate(total_df['image_filename']):
#     try : 
#         total_df.at[i,"image_vec"]= get_vector(*file).astype(object)
#     except : 
#         pass
    
# total_df

In [11]:
#벡터화 불가능
#에러 :RuntimeError: output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]
# removeL = np.where(total_df['image_vec'].isin([0]))
# for i in removeL:
#     total_df.drop(i, inplace=True)

In [12]:
# #변환 가능한 이미지만 담긴 데이터프레임 저장
# total_df.to_csv("/Users/hyunji/2023 Conference/temp_imagevector.csv", index = False)

In [13]:
# def check_vector(vector):
#     return np.array([0.0 if str(x) == '' else float(x) for x in vector])[:512]


# total_df=pd.read_csv("/Users/hyunji/2023 Conference/temp_imagevector.csv",
#                  converters={"image_vec": lambda x: x.strip("[]").replace('\n','').split(" ")})
# total_df['image_vec'] = total_df['image_vec'].apply(lambda x: check_vector(x))
# total_df.drop(columns = ["Unnamed: 0"], inplace=True)

## Kor2vec

In [14]:
# # kor2vec_hyperparameter 조정
# embed_size_tune = 64
# batch_size_tune = 64
# seq_length = 20

# class preprocess(Dataset):
#     def to_list(x):
#         return ast.literal_eval(x)
    
#     def max_len(x):
#         return len(x)

#     def split_(x):
#         return x.replace(',',' ')
    
#     total_df['tag'] = total_df['tag'].apply(split_)

    
#     if kor_vec_name not in os.listdir(model_dir):

#     # 태그 데이터를 사용해 임베딩을 학습 실시

#         rev = pd.Series("<sos> " + total_df['tag']+" <end>")
#         rev.to_csv(model_dir+"summary_text_corpus.csv")

#         # skip-gram 기반 임베딩 training
#         kor2vec = Kor2Vec(embed_size=embed_size_tune) # embed_size : 임베딩 벡터의 2번째 차원(차원 수)
#         kor2vec.train(model_dir+"summary_text_corpus.csv", 'model.kor2vec', batch_size=batch_size_tune) # 임베딩 실시 (학습)
#         kor2vec.save(model_dir+ kor_vec_name)  # 임베딩 모델 저장

Reading Corpus lines


Corpus Sampling: 100%|██████████| 44175/44175 [00:00<00:00, 121508.05it/s]


Training kor2vec
Loading Word_sample corpus
Loading corpus finished
CUDA Available/count: True 1
training on  cuda:0


EP 0: 100%|██████████| 2418/2418 [00:12<00:00, 196.38it/s]


{'epoch': 0, 'train_ep_loss': 0.540484758933607}


...

{'epoch': 7, 'train_ep_loss': 0.4319451482176189}


EP 8: 100%|██████████| 2418/2418 [00:12<00:00, 198.60it/s]


{'epoch': 8, 'train_ep_loss': 0.42727211975269913}


EP 9: 100%|██████████| 2418/2418 [00:12<00:00, 196.69it/s]

{'epoch': 9, 'train_ep_loss': 0.4246300945075412}


In [15]:
# total_df

,Unnamed: 0,product_num,product_name,brand,year_sold,like,rate,price,tag,categories,image_filename
0,0,2861160,데일리 코듀로이 밴딩 팬츠 - 4 COLOR,SUARE,1.6만 개 이상,"19,150",4.8,"59,000원",바지 팬츠 와이드팬츠 밴딩팬츠 코듀로이 코튼팬츠 캐주얼팬츠,cotton,[/home/augustin/project/bitamin-conference/dat...
1,1,957401,P[1003]롱 와이드 코튼팬츠(베이지),PUBLIC ICON,1.8만 개 이상,"55,445",4.7,"68,000원",데님팬츠 와이드면팬츠 와이드코튼팬츠 와이드팬츠 와이드 베이지,cotton,[/home/augustin/project/bitamin-conference/dat...
2,2,2060759,[기모선택] 와이드 절개 벌룬팬츠 코튼 카키,GLW,1.6만 개 이상,"35,679",4.9,"74,800원",와이드팬츠 코튼팬츠 힙합 와이드핏 하이웨이스트 오버핏 아메카지,cotton,[/home/augustin/project/bitamin-conference/dat...
3,3,1627892,레플리카 퍼티그 팬츠 [카키],MUSINSA STANDARD,1.6만 개 이상,"14,833",4.9,"43,900원",밀리터리 레플리카 군용 포켓 백사틴 바지 면바지 캐주얼팬츠 유아인착용상품,cotton,[/home/augustin/project/bitamin-conference/dat...
4,4,2183795,에센셜 세미 와이드 코듀로이 팬츠 크림,GLW,1.1만 개 이상,"21,312",4.8,"49,000원",와이드팬츠 겨울바지 골덴팬츠 골덴바지 면바지 남자오버핏 코듀로이팬츠,cotton,[/home/augustin/project/bitamin-conference/dat...
...,...,...,...,...,...,...,...,...,...,...,...
44169,6082,1602281,해리 피그먼트 워싱 맨투맨 카키 OYMT9058,UNDERBASE,50개 이상,111,4.7,"37,900원",스웨트셔츠 맨투맨 오버핏 피그먼트 로고맨투맨 오버핏맨투맨,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44170,6083,1580282,MARITHE OVAL LOGO SWEATSHIRT black,MARITHE FRANCOIS GIRBAUD,50개 미만,111,4.6,"79,000원",맨투맨 스웻셔츠 로고맨투맨 스웻 MFG맨투맨 엠에프지맨투맨 스웨트셔츠,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44171,6084,1319747,엠엘 모어 이즈 레스 레터링 크루넥 스웨트 [멜란지],THEMILE,50개 미만,44,4.9,"44,600원",데일리 맨투맨 오버핏 커플 크루넥,sweatshirt,[/home/augustin/project/bitamin-conference/dat...
44172,6085,1241808,토트넘맨투맨 TTNRMA03 [BLACK],TOTTENHAM APPAREL,50개 미만,49,4.6,"49,000원",나라물산 맨투맨 오버핏 토트넘손흥민,sweatshirt,[/home/augustin/project/bitamin-conference/dat...


# filename preprocess

In [16]:

total_df["image_filename"] = total_df["image_filename"].astype('str').apply(lambda x : x.replace('[',""))
total_df["image_filename"] = total_df["image_filename"].astype('str').apply(lambda x : x.replace(']',""))
total_df["image_filename"] = total_df["image_filename"].astype('str').apply(lambda x : x.replace("'",""))

In [17]:
total_df

,Unnamed: 0,product_num,product_name,brand,year_sold,like,rate,price,tag,categories,image_filename
0,0,2861160,데일리 코듀로이 밴딩 팬츠 - 4 COLOR,SUARE,1.6만 개 이상,"19,150",4.8,"59,000원",바지 팬츠 와이드팬츠 밴딩팬츠 코듀로이 코튼팬츠 캐주얼팬츠,cotton,/home/augustin/project/bitamin-conference/data...
1,1,957401,P[1003]롱 와이드 코튼팬츠(베이지),PUBLIC ICON,1.8만 개 이상,"55,445",4.7,"68,000원",데님팬츠 와이드면팬츠 와이드코튼팬츠 와이드팬츠 와이드 베이지,cotton,/home/augustin/project/bitamin-conference/data...
2,2,2060759,[기모선택] 와이드 절개 벌룬팬츠 코튼 카키,GLW,1.6만 개 이상,"35,679",4.9,"74,800원",와이드팬츠 코튼팬츠 힙합 와이드핏 하이웨이스트 오버핏 아메카지,cotton,/home/augustin/project/bitamin-conference/data...
3,3,1627892,레플리카 퍼티그 팬츠 [카키],MUSINSA STANDARD,1.6만 개 이상,"14,833",4.9,"43,900원",밀리터리 레플리카 군용 포켓 백사틴 바지 면바지 캐주얼팬츠 유아인착용상품,cotton,/home/augustin/project/bitamin-conference/data...
4,4,2183795,에센셜 세미 와이드 코듀로이 팬츠 크림,GLW,1.1만 개 이상,"21,312",4.8,"49,000원",와이드팬츠 겨울바지 골덴팬츠 골덴바지 면바지 남자오버핏 코듀로이팬츠,cotton,/home/augustin/project/bitamin-conference/data...
...,...,...,...,...,...,...,...,...,...,...,...
44169,6082,1602281,해리 피그먼트 워싱 맨투맨 카키 OYMT9058,UNDERBASE,50개 이상,111,4.7,"37,900원",스웨트셔츠 맨투맨 오버핏 피그먼트 로고맨투맨 오버핏맨투맨,sweatshirt,/home/augustin/project/bitamin-conference/data...
44170,6083,1580282,MARITHE OVAL LOGO SWEATSHIRT black,MARITHE FRANCOIS GIRBAUD,50개 미만,111,4.6,"79,000원",맨투맨 스웻셔츠 로고맨투맨 스웻 MFG맨투맨 엠에프지맨투맨 스웨트셔츠,sweatshirt,/home/augustin/project/bitamin-conference/data...
44171,6084,1319747,엠엘 모어 이즈 레스 레터링 크루넥 스웨트 [멜란지],THEMILE,50개 미만,44,4.9,"44,600원",데일리 맨투맨 오버핏 커플 크루넥,sweatshirt,/home/augustin/project/bitamin-conference/data...
44172,6085,1241808,토트넘맨투맨 TTNRMA03 [BLACK],TOTTENHAM APPAREL,50개 미만,49,4.6,"49,000원",나라물산 맨투맨 오버핏 토트넘손흥민,sweatshirt,/home/augustin/project/bitamin-conference/data...


# 데이터 로더에 들어가기 전 전처리 완료된 데이터프레임 저장

In [46]:
total_df.to_csv("/home/augustin/project/bitamin-conference/recsys/dataframe/to_Dataloader.csv", index = False)

: 